In [1]:
import pandas as pd
import json
from tqdm.auto import tqdm

import gzip
from collections import defaultdict
from datetime import datetime

In [ ]:
meta_name = 'meta_Sports_and_Outdoors'

In [2]:
# meta 데이터 셋
%%time
data = []
with gzip.open(f'data/Amazon_2018/{meta_name}.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

meta = pd.DataFrame.from_dict(data)

print('원래 사이즈 :', pd.DataFrame.from_dict(data).shape)

CPU times: user 41.5 s, sys: 2.44 s, total: 44 s
Wall time: 44 s


In [5]:
# meta 데이터 중 description 없는 데이터 셋 제거
drop_li = []
for n, i in enumerate(meta['description']):
    if len(i) != 0 :
        drop_li.append(n)

meta = meta.iloc[drop_li].reset_index(drop = True)

In [7]:
# meta 데이터 중 imageURL 없는 데이터 셋 제거
drop_li = []
for n, i in enumerate(meta['imageURL']):
    if len(i) != 0 :
        drop_li.append(n)

meta = meta.iloc[drop_li].reset_index(drop = True)

In [ ]:
data_name = 'Sports_and_Outdoors_5'

In [9]:
# 5-core 데이터 셋
%%time
data = []
with gzip.open(f'data/Amazon_2018/{data_name}.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

CPU times: user 33.9 s, sys: 712 ms, total: 34.6 s
Wall time: 34.6 s


In [10]:
origin = pd.DataFrame.from_dict(data)
print(origin.shape)

In [21]:
# 데이터셋 구성
countU = defaultdict(lambda: 0)
countP = defaultdict(lambda: 0)
line = 0

for i in range(len(origin)) : 
    asin = origin['asin'][i]
    rev  = origin['reviewerID'][i]
    time = origin['unixReviewTime']
    countU[rev] += 1
    countP[asin] += 1

In [23]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
User = dict()

for i in tqdm(range(len(origin))) : 
    asin = origin['asin'][i]
    rev  = origin['reviewerID'][i]
    time = origin['unixReviewTime']
    
    if countU[rev] < 5:
        continue
    
    if rev in usermap:
        userid = usermap[rev]
    else:
        usernum += 1
        userid = usernum
        usermap[rev] = userid
        User[userid] = []
    
    if asin in itemmap:
        itemid = itemmap[asin]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[asin] = itemid
    User[userid].append([time, itemid])
# sort reviews in User according to time    

In [25]:
print(usernum, itemnum)

153940 55697


In [26]:
f = open('data/Amazon_2018/Sports_and_Outdoors.txt', 'w')
for user in User.keys():
    for i in User[user]:
        f.write('%d %d\n' % (user, i[1]))
f.close()

In [27]:
meta = meta[meta['asin'].isin(list(itemmap.keys()))].reset_index(drop = True)

In [29]:
for n, i in tqdm(enumerate(meta['asin'])):
    meta['asin'][n] = itemmap[i]

0it [00:00, ?it/s]

### Extract text features

In [50]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch

from tqdm.auto import tqdm

2022-06-07 09:40:00.448941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-06-07 09:40:00.448960: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [51]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [52]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [53]:
def bert_text_preparation(text, tokenizer):
    indexed_tokens = tokenizer(text[0], truncation = True).input_ids
    segments_ids = [1]*len(indexed_tokens)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    return tokens_tensor, segments_tensors

In [54]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)

    return outputs[1]

In [55]:
outputs = []
for text in tqdm(meta['description']) :
    tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    output = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    outputs.append(output.detach().numpy()[0])

  0%|          | 0/56187 [00:00<?, ?it/s]

In [56]:
np.save('data/Amazon_2018/pre_description_Sports_and_Outdoors', outputs)

### Extract image features

In [30]:
import numpy as np

import urllib
from urllib.request import urlopen
from urllib.error import URLError, HTTPError

import cv2
from PIL import Image

import torch
from torchvision import models
from torchvision import transforms

In [34]:
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    return image

In [73]:
model = models.vgg16(pretrained=True)
model.classifier = model.classifier[:-1]

In [48]:
%%time
image_features = []
no_url_list = [] # Url 주소가 존재하나 간혹 웹 오류로 인한 url 접근 불가 상황 발생

for n, url in tqdm(enumerate(meta['imageURLHighRes'])) :
    try : 
        image = Image.fromarray(url_to_image(url[0]))
        tensor = transforms.ToTensor()(transforms.Resize((224, 224))(image)).unsqueeze(dim = 0)
        image_features.append(model(tensor).detach().numpy()[0])
    except : 
        try : 
            image = Image.fromarray(url_to_image(url[-1]))
            tensor = transforms.ToTensor()(transforms.Resize((224, 224))(image)).unsqueeze(dim = 0)
            image_features.append(model(tensor).detach().numpy()[0])
        except : 
            print(n)
            no_url_list.append(n)

0it [00:00, ?it/s]

2269
21174
CPU times: user 12h 42min 31s, sys: 7min 29s, total: 12h 50min 1s
Wall time: 3h 10min 3s


In [68]:
# 2269번 이미지 추가
err_image = Image.fromarray(url_to_image(meta['imageURLHighRes'][2269][1]))
err_tensor = transforms.ToTensor()(transforms.Resize((224, 224))(err_image)).unsqueeze(dim = 0)
err_vector = model(err_tensor).detach().numpy()[0]
image_features.insert(2269, err_vector)

In [ ]:
# 21174번 이미지 추가
err_image2 = Image.fromarray(url_to_image(meta['imageURLHighRes'][21174][4]))
err_tensor2 = transforms.ToTensor()(transforms.Resize((224, 224))(err_image2)).unsqueeze(dim = 0)
err_vector2 = model(err_tensor2).detach().numpy()[0]
image_features.insert(21174, err_vector2)

In [111]:
np.save('data/Amazon_2018/pre_image_Sports_and_Outdoors', image_features)

In [112]:
image = np.load('data/Amazon_2018/pre_image_Sports_and_Outdoors.npy')
text = np.load('data/Amazon_2018/pre_description_Sports_and_Outdoors.npy')

In [113]:
len(image)

56187

In [115]:
len(text)

56187